In [1]:
import pandas, sqlalchemy

In [3]:
file = pandas.read_excel('/home/hrobbin/python/mysql_scripts/JivoChat Report 21-09-02 61305d4165b6d.xlsx', sheet_name=0)

In [37]:
res1 = file[file.isin(['Сводный отчет по сайтам', 'ИТОГО']).any(1)]
res1

,Отчеты JivoSite,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12
4,Сводный отчет по сайтам,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16,ИТОГО,12,0,1,0,1,1,NaN,NaN,NaN,NaN,NaN,NaN
39,ИТОГО,12,0,0,0,1,1,19:09:43,00:38:55,161,00:31:52,0,1
67,ИТОГО,1,0,0,0,0,0,11:52:05,00:31:28,432,00:49:26,0,0
81,ИТОГО,0,0,0,0,0,0,11:52:05,00:31:28,0,00:00:00,0,0
94,ИТОГО,10,0,0,0,1,0,11:52:05,00:31:28,134,00:26:11,0,1
107,ИТОГО,0,0,0,0,0,0,11:52:05,00:31:28,0,00:00:00,0,0
120,ИТОГО,0,0,0,0,0,0,11:52:05,00:31:28,0,00:00:00,0,0
133,ИТОГО,0,0,0,0,0,0,11:52:05,00:31:28,0,00:00:00,0,0
146,ИТОГО,1,0,0,0,0,1,11:52:05,00:31:28,0,01:16:46,0,0
